## CA 4, LLMs Spring 2024

- **Name:** Mohammad Mahdi Salmani
- **Student ID:** 810102174

---
### This is due on **14 June**, submitted via [elearn](https://elearn.ut.ac.ir/).
#### Your submission should be named using the following format: `CA4_LASTNAME_STUDENTID.ipynb`.

---

##### *How to do this problem set:*

- Some questions require writing Python code and computing results, and the rest of them have written answers. For coding problems, you will have to fill out all code blocks that say `WRITE YOUR CODE HERE`.

- For text-based answers, you should replace the text that says "Write your answer here..." with your actual answer.

- There is no penalty for using AI assistance on this homework as long as you fully disclose it in the final cell of this notebook (this includes storing any prompts that you feed to large language models). That said, anyone caught using AI assistance without proper disclosure will receive a zero on the assignment (we have several automatic tools to detect such cases). We're literally allowing you to use it with no limitations, so there is no reason to lie!

---

##### *Academic honesty*

- We will audit the Colab notebooks from a set number of students, chosen at random. The audits will check that the code you wrote actually generates the answers in your notebook. If you turn in correct answers on your notebook without code that actually generates those answers, we will consider this a serious case of cheating.

- We will also run automatic checks of Colab notebooks for plagiarism. Copying code from others is also considered a serious case of cheating.

---

# RLHF (55 points)

## Introduction to RLHF

<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2023/08/31/ML-14874_image001.jpg"/>
</div>

With the recent public introduction of ChatGPT, reinforcement learning from human feedback (RLHF) has become a hot topic in language modeling circles -- both academic and industrial.
We can trace the application of RLHF to natural language processing OpenAI's 2019 release of <br>[Fine-Tuning Language Models from Human Preferences](https://arxiv.org/abs/1909.08593).

Fast forward one year when OpenAI released one of its first significant papers on reinforcement learning from human feedback applied to natural language generation.

In that paper-<br>[Learning to summarize from human feedback](https://arxiv.org/abs/2009.01325)-OpenAI showed that simply fine-tuning on summarization data leads to suboptimal performance when evaluated on human preferences. The authors suggest optimizing for human preferences directly via a reinforcement learning approach to alleviate these performance issues.


**Learn More:**
<br>[Huggingface Deep Reinforcement Learning Course](https://huggingface.co/learn/deep-rl-course/en/unit0/introduction)
<br>[Research Papers for Reinforcement Learning with Human Feedback ](https://github.com/opendilab/awesome-RLHF)


## Import Libraries and Set Constants

In [ ]:
%pip install datasets
%pip install evaluate
%pip install rouge_score
%pip install accelerate -U
%pip install transformers[torch]

In [1]:
import numpy as np
import pandas as pd
import json
import random
import evaluate
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_scheduler
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
)
from transformers import AdamW
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
class CONFIG:
    seed = 42
    max_len = 550
    train_batch_size = 16
    eval_batch_size = 1
    eval_steps = 500
    epochs = 5
    save_steps = 1000
    learning_rate = 1e-4
    gradient_accumulation_steps = 1
    model_name = 'gpt2'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    output_dir = "gpt2-supervised-summarize-checkpoint"
    output_dir_rm = "rm_checkpoint"

device = CONFIG.device
rw_device = CONFIG.device

## Implementing Learning for Summarization

 In this notebook by using trlX, we will implement RLHF for a summarization task. The training process consists of three parts:

*   We will first fine-tune a pre-trained transformer model on our summarization dataset. This is our supervised fine-tuned model (SFT).
* We will then train a reward model (RM). This model is initialized from the SFT model and outputs a scalar value. This scalar value is the reward that indicates the preferability of a summary.  

*   Finally, we use the RM to fine-tune the SFT model via PPO. This step aligns our SFT model with human preference.

## Section One: Supervised Fine Tuning (5 points)

### Dataset

For our experiment, we'll use the **TLDR summarization** dataset used originally in Learning to summarize from human feedback.

Based on that training process described above, we'll need two types of datasets:

*   One for fine-tuning the pre-trained supervised model and then for fine-tuning it again with PPO and reward model, and
*   One for training our reward model.

In our case, the dataset for fine-tuning is the filtered* TLDR dataset. The dataset for training our reward model is the **comparison or preference dataset**.

**Note:** I set the number of training examples to 6000, you can increase it, also you can adjust the number of validation examples.


In [3]:
tlrdataset_path = "CarperAI/openai_summarize_tldr"
comparissions_path = "CarperAI/openai_summarize_comparisons"

#### Create Dataset

In [4]:
class TLDRDataset(Dataset):
    def __init__(self, path, tokenizer, split, max_length=CONFIG.max_len):
        self.post_list = []
        dataset = load_dataset(path, split=split)
        for sample in dataset:
            self.post_list.append(sample["prompt"] + sample["label"])

        if "train" in split:
          self.post_list = random.sample(self.post_list, min(6000, len(self.post_list)))
        elif "valid" in split:
            self.post_list = self.post_list[0:2000]

        self.tokenizer = tokenizer
        self.max_length = max_length
        self.input_ids = []
        self.attn_masks = []

    def __len__(self):
        return len(self.post_list)

    def __getitem__(self, idx):
        txt = self.post_list[idx]
        encodings_dict = self.tokenizer(txt, truncation=True, max_length=self.max_length, padding="max_length")
        input_ids = torch.tensor(encodings_dict["input_ids"])
        attn_masks = torch.tensor(encodings_dict["attention_mask"])

        return {
            "input_ids": input_ids,
            "attention_mask": attn_masks,
            "labels": input_ids,
        }

#### Load Dataset

In [5]:
train_dataset = TLDRDataset(
  path=tlrdataset_path,
  tokenizer=CONFIG.tokenizer,
  split="train")

dev_dataset = TLDRDataset(
  path=tlrdataset_path,
  tokenizer=CONFIG.tokenizer,
  split="valid")

### Load Model and Tokenizer

In [6]:
#Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(CONFIG.model_name, use_cache=False)
tokenizer = CONFIG.tokenizer

# Setting pad token to eos token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# Resize token embeddings
model.resize_token_embeddings(len(tokenizer))

# Update model configuration
model.config.pad_token_id = tokenizer.pad_token_id
model.config.end_token_id = tokenizer.eos_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

### Define Compute metric function (2.5 Points)

In this part, you should implement an evaluation function that computes rouge scores for our predicted summaries.

In [7]:
# Load the ROUGE metric
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True) for pred in preds]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True, clean_up_tokenization_spaces=True) for label in labels]
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return result

In [8]:
# Create a preprocessing function to extract out the proper logits from the model output
def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits.argmax(dim=-1)

### Train

In [9]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir=CONFIG.output_dir,
    evaluation_strategy="steps",
    eval_steps=250,
    save_steps=500,
    eval_accumulation_steps=1,
    learning_rate=CONFIG.learning_rate,
    per_device_train_batch_size=32, # I increased the batch_size
    per_device_eval_batch_size=32,
    gradient_checkpointing=True,
    fp16=True,
    fp16_backend="auto",
    adam_beta1=0.9,
    adam_beta2=0.95,
    num_train_epochs=4, # First I train the model for one epoch, then I train for four more apochs
    warmup_steps=100,
    load_best_model_at_end=True,
    logging_dir='./logs',
    logging_steps=500 #TODO
)

In [34]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,
    data_collator=default_data_collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)

In [35]:
torch.cuda.empty_cache()

In [36]:
# Start training
trainer.train()

# Save the model
trainer.save_model(CONFIG.output_dir)

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
250,1.898400,1.923292,0.582104,0.180484,0.381413,0.504274
500,1.875600,1.912207,0.586530,0.182838,0.383275,0.509494
750,1.843300,1.908619,0.586853,0.183296,0.384129,0.509715


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


In [37]:
trainer.evaluate()

{'eval_loss': 1.9122065305709839,
 'eval_rouge1': 0.5865301790188278,
 'eval_rouge2': 0.1828376831114582,
 'eval_rougeL': 0.383274792047039,
 'eval_rougeLsum': 0.509493881204031,
 'eval_runtime': 128.17,
 'eval_samples_per_second': 15.604,
 'eval_steps_per_second': 0.492,
 'epoch': 4.0}

### Test (2.5 Points)

Report rouge scores for test set of TLDR dataset.

In [38]:
test_dataset = TLDRDataset(
    path=tlrdataset_path,
    tokenizer=CONFIG.tokenizer,
    split="test"
)

In [39]:
test_results = trainer.evaluate(test_dataset)
print("Test ROUGE scores:")
test_results

Test ROUGE scores:


{'eval_loss': 1.9079687595367432,
 'eval_rouge1': 0.5875083782980206,
 'eval_rouge2': 0.1831651352260253,
 'eval_rougeL': 0.3838607240070928,
 'eval_rougeLsum': 0.509185929201362,
 'eval_runtime': 426.4569,
 'eval_samples_per_second': 15.366,
 'eval_steps_per_second': 0.481,
 'epoch': 4.0}

## Section Two: Reward Model Training (25 Points)

### Reward Model

We'll initialize the reward model from the SFT model and attach a randomly initialized linear head that outputs a scalar value on top.

Next, we'll dig into how the data is input to the model, the loss function, and other gotchas of a reward model in more detail.

### Question 1 (2.5 Points)

**How would you create a comparison dataset for a text summarization task? (explain the entire procedure)**

This thask involves several key steps: data collection, preprocessing, initial summarization, human annotation, and dataset structuring. Here’s a detailed guide:

**1. Data Collection**
- Source Selection: Choose diverse sources like news articles, research papers, or blogs.
- Data Gathering: Collect a substantial number of documents through web scraping, APIs, or open datasets.

**2. Preprocessing**
- Text Cleaning: Remove unwanted characters and normalize the text.
- Segmentation: Break documents into smaller units if they are too long.

**3. Initial Summarization**
- Automatic Summarization: Use the model to generate initial summaries for each document.

**4. Human Annotation**
- Annotation Guidelines: Develop clear guidelines for comparing summaries on relevance, coherence, conciseness, readability, and accuracy.
- Annotator Training: Train annotators to apply guidelines consistently.
- Pairwise Comparison: Present annotators with pairs of summaries (e.g., machine vs. human-written) and ask them to rate or rank them.
- Rating and Ranking: Collect ratings or rankings based on the provided guidelines.

**5. Dataset Creation**
- Include the document, summary pairs, and annotations. Ensure a balanced dataset with diverse examples.
- Check for consistent and reliable annotations through cross-validation and reviews.

**6. Post-processing**
- Normalize scores or rankings for consistency.
- Format the dataset to meet the requirements of your training framework.

**7. Reward Model Integration**
- Initialize Reward Model: Start with the supervised fine-tuned (SFT) summarization model.
- Add a Linear Head: Attach a randomly initialized linear layer to the model that outputs a scalar value for scoring summaries.


Creating a comparison dataset involves collecting and preprocessing data, generating initial summaries, obtaining human annotations through pairwise comparisons, structuring and formatting the dataset, and performing quality control. Finally, integrate the reward model by initializing it from the SFT model and adding a linear head for evaluation. This dataset and model setup are essential for training in a reinforcement learning from human feedback (RLHF) framework.

### Question 2 (2.5 Points)

**If you have 100 pairs of summaries, and for each pair one summary is prefered, how would you structurre your training data for the reward model?**

**Data Structure Definition**

Each entry should include:
* The document text.
* The chosen summary.
* The rejected summary.


**Data Processing**

Pairwise Data Extraction:
* Extract pairs of summaries from the dataset.
* Assign them to "chosen" and "rejected" based on the preference.


```python
# Assume we have a list of comparisons
comparisons = [
    {
        "prompt": "Full text of document 1",
        "summary_1": "Generated summary 1",
        "summary_2": "Generated summary 2",
        "label": 0 # Assign wich summary is prefered
    },
    {
        "prompt": "Full text of document 2",
        "summary_1": "Generated summary 3",
        "summary_2": "Generated summary 4",
        "label": 1 # Assign wich summary is non-prefered
    }
]

# Prepare data for training
train_data = []
for comparison in comparisons:
    document = comparison["prompt"]
    summary_1 = comparison["summary_1"]
    summary_2 = comparison["summary_2"]
    preferred_summary = comparison["label"]
    
    if preferred_summary == 0:
        train_data.append({
            "prompt": document,
            "chosen": summary_1,
            "rejected": summary_2
        })
    else:
        train_data.append({
            "prompt": document,
            "chosen": summary_2,
            "rejected": summary_1
        })

```

Finally we have this dataset:


```json
[
    {
        "prompt": "Full text of document 1",
        "chosen": "prefered summary",
        "rejected": "non prefered summary"
    },
    {
        "prompt": "Full text of document 2",
        "chosen": "prefered summary",
        "rejected": "non prefered summary"
    }
]
```

### Raw Input

Now, we'll create a list of dicts using the create_comparison_dataset function (shown below), where each dict has two keys - chosen and rejected. The value of each key is the prompt (or Reddit post) concatenated with the summary.

**Note:** You can increase the number of training examples.

In [7]:
def create_comparison_dataset(
     path="CarperAI/openai_summarize_comparisons", split="train"
 ):
     dataset = load_dataset(path, split=split)
     if split == "test":
         dataset = dataset.select(range(1000))
     elif split == "train":
         dataset = dataset.select(range(10000))

     pairs = []
     for sample in tqdm(dataset):
         pair = {}
         prompt = sample["prompt"]
         chosen_summary = sample["chosen"]
         rejected_summary = sample["rejected"]
         if chosen_summary == rejected_summary:
             continue
         if  len(chosen_summary.split()) < 5 or len(rejected_summary.split()) < 5:
             continue
         pair["chosen"] = prompt + "\n" + chosen_summary
         pair["rejected"] = prompt + "\n" + rejected_summary
         pairs.append(pair)
     return pairs


### Pairwise Dataloader (2.5 points)

The PairwiseDataset class shown below tokenizes the chosen and rejected "summaries". The dataset class return the input_ids and attention_masks for both chosen and rejected summaries, in this part you should complete the **PairwiseDataset class.**

In [11]:
class PairwiseDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_length=512):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]

        chosen_encodings = self.tokenizer(pair["chosen"], truncation=True, max_length=self.max_length, padding="max_length", return_tensors="pt")
        rejected_encodings = self.tokenizer(pair["rejected"], truncation=True, max_length=self.max_length, padding="max_length", return_tensors="pt")

        return [{
            "chosen_input_ids": chosen_encodings["input_ids"].squeeze(),
            "chosen_attention_mask": chosen_encodings["attention_mask"].squeeze(),
            "rejected_input_ids": rejected_encodings["input_ids"].squeeze(),
            "rejected_attention_mask": rejected_encodings["attention_mask"].squeeze()
        }]

### Data Collator (2.5 Points)

The DataCollatorReward class creates batches (dict) of data for our reward model. The collator returns:

*   input_ids: collator concatenates the chosen and rejected summaries' input_ids across dim=0.
*   attention_mask: collator concatenates the chosen and rejected summaries' attention_mask across dim=0.

*   labels: collator creates a tensor of zeros for chosen summaries and a tensor of ones for rejected summaries concatenated across dim=0.

In [12]:
class DataCollatorReward:
    def __call__(self, data):
        batch = {}

        chosen_input_ids = torch.stack([item[0]["chosen_input_ids"] for item in data])
        rejected_input_ids = torch.stack([item[0]["rejected_input_ids"] for item in data])
        
        chosen_attention_mask = torch.stack([item[0]["chosen_attention_mask"] for item in data])
        rejected_attention_mask = torch.stack([item[0]["rejected_attention_mask"] for item in data])
        
        # Concatenate chosen and rejected summaries
        batch["input_ids"] = torch.cat((chosen_input_ids, rejected_input_ids), dim=0)
        batch["attention_mask"] = torch.cat((chosen_attention_mask, rejected_attention_mask), dim=0)

        # Create labels: 0 for chosen summaries, 1 for rejected summaries
        chosen_labels = torch.zeros(chosen_input_ids.size(0), dtype=torch.long)
        rejected_labels = torch.ones(rejected_input_ids.size(0), dtype=torch.long)
        batch["labels"] = torch.cat((chosen_labels, rejected_labels), dim=0)

        return batch

### What is happening in reward model?

Here, we have a Reddit post and two summaries (<font color='green'><b>chosen</b></font> and <font color='red'><b>rejected</b></font>) as input.

The ground truth label (**labels**) is the human feedback (<font color='green'><b>0 for chosen</b></font> and <font color='red'><b>1 for rejected</b></font>). And the loss function (pairwise ranking loss) is given as:

$$\text{loss}(r_{\theta}) = -\mathbb{E}_{(x, y_0, y_1, i) \sim D} \left[ \log \left( \sigma \left( r_{\theta}(x, y_i) - r_{\theta}(x, y_{1-i}) \right) \right) \right]
$$.


where:
- $ x $ is the post,
- $ y_0 $ and $ y_1 $ are the summaries,
- $ i $ in {0, 1} indicates which summary is preferred by humans,
- $ r_{\theta}(x, y) $ is the reward model that returns a scalar value for the post $ x $ and the summary $ y $,
- $ \sigma $ is the sigmoid function.


The reward model $ r_{\theta} $ takes the post $ x $ and the summary $ y $ and returns a scalar value. The value is computed for both summaries and a sigmoid activation is applied to the difference.

Finally, the negative log is computed.

This loss function encourages the model to give higher scores to human-preferred summaries.

**How to code this?**

Our model receives input prepared by the data collator.

*   This input is passed through the GPT-2 model to get the final hidden states.

*   The hidden state is then passed through the linear layer to get a reward score.

*   For each batch fed into the model, the first half is the chosen summaries, and the second half is the rejected summaries.

*   The forward method of the model iterates through each input sample to compute pairwise loss.
*  Return loss and chosen summaries scores and rejected summaries scores.

### Question 3 (2.5 Points)

**What is the goal of pairwise ranking loss? and how we achieve this goal?**

**Goal of Pairwise Ranking Loss**

The goal of the pairwise ranking loss in the context of training a reward model is to learn a scoring function $ r_{\theta} $ that assigns higher scores to preferred outputs compared to less preferred outputs for the same input. In other words, the reward model should learn to rank the preferred (chosen) summaries higher than the rejected ones.

**Achieving the Goal**

To achieve this goal, the loss function operates as follows:

1. **Score Calculation**:
   - The reward model $ r_{\theta} $ computes scores for both the chosen summary $ y_0 $ and the rejected summary $ y_1 $.

2. **Score Difference**:
   - Calculate the difference in scores: $ r_{\theta}(x, y_i) - r_{\theta}(x, y_{1-i}) $. Here, $ y_i $ is the preferred summary, and $ y_{1-i} $ is the less preferred one.

3. **Sigmoid Function**:
   - Apply the sigmoid function to this difference. The sigmoid function $ \sigma(z) $ maps the difference to a probability range (0, 1), representing the <ins>likelihood</ins> that the chosen summary $ y_i $ is better than the rejected summary $y_{1-i}$.

4. **Log Probability**:
   - Take the logarithm of this probability. The log probability penalizes the model more heavily for being confident but wrong.

5. **Expectation**:
   - The expectation $ \mathbb{E}_{(x, y_0, y_1, i) \sim D} $ is taken over all pairs in the dataset, aiming to minimize this expected loss.

Now, For each input $x$:
1. We generate scores for both chosen and rejected summaries using the reward model.
2. Compute the pairwise ranking loss using the formula.
3. Backpropagate the loss and update the model parameters $\theta$ to minimize the loss.

=> minimizing the $loss$ => assigns higher scores to preferred outputs and lower scores to non-preferred for the same input.

### Question 4 (2.5 Points)

**Explain that in the process of training the reward model, how the pairwise ranking loss can avoid the problem of huge score difference between the answers (summaries) and why is this useful?**

**Avoiding Huge Score Differences:**

**1. Sigmoid Function**

The sigmoid function $ \sigma(z) = \frac{1}{1 + e^{-z}} $ maps the input $ z $ to a value between 0 and 1. This function has a property where:
- Small differences between $ r_{\theta}(x, y_i) $ and $ r_{\theta}(x, y_{1-i}) $ produce values close to 0.5.
- Large positive differences produce values close to 1.
- Large negative differences produce values close to 0.

**2. Log Probability**

Taking the logarithm of the sigmoid output further stabilizes the learning process. The logarithm function compresses the range of values, ensuring that extremely confident predictions (whether correct or incorrect) do not dominate the loss computation excessively. This avoids scenarios where huge score differences could lead to very large or very small gradient updates, which could destabilize training.

**Why This Is Useful:**

**1. Stable Training**

Without this bounding, the gradients could become too large or too small, leading to issues like exploding or vanishing gradients. This stability allows the model to learn more effectively across iterations.

**2. Focus on Relative Differences**

The pairwise ranking loss focuses on the relative ranking of the chosen and rejected summaries rather than their absolute scores. This is particularly useful because:
- It aligns with the goal of ranking preferred summaries higher, regardless of the actual score magnitudes.
- It allows the model to learn from the ranking order rather than being affected by absolute score values, which can vary widely.

**3. Robustness to Outliers**

Outliers or extremely different summaries can skew the training process if absolute score differences are used directly. By focusing on the relative difference and using the sigmoid function, the model becomes more robust to such outliers, preventing them from disproportionately influencing the training.

### Implementing The Reward Model (5 Points)

In [52]:
class GPTRewardModel(nn.Module):
    def __init__(self, model_path, model=None):
        super().__init__()
        if model is not None:
            self.model = model
        else:
            self.model = AutoModelForCausalLM.from_pretrained(model_path)
        self.config = self.model.config
        self.linear = nn.Linear(self.config.hidden_size, 1)
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        print(f'Outputs logits: {outputs.logits.shape}')
        print(f'Outputs hid.state: {len(outputs.hidden_states)}')
        print(f'Outputs hid.state: {outputs.hidden_states[-1].shape}')
        hidden_states = outputs.hidden_states[-1]  # Get the last hidden state
        mean_hidden_state = hidden_states.mean(dim=1)  # Average over the sequence length
        reward = self.linear(mean_hidden_state).squeeze(-1)

        if labels is not None:
            loss = self.loss_fn(reward, labels.float())
            return loss, reward

        return reward

    def save_pretrained(self, save_directory):
        self.model.save_pretrained(save_directory)
        torch.save(self.linear.state_dict(), f"{save_directory}/linear_layer.bin")

    @classmethod
    def from_pretrained(cls, load_directory):
        model_path = load_directory
        model = cls(model_path)
        model.model = AutoModelForCausalLM.from_pretrained(load_directory)
        model.linear.load_state_dict(torch.load(f"{load_directory}/linear_layer.bin"))
        return model

**After finishing the above code, could you please explain how the scores for the selected summaries and the scores for the rejected summaries are calculated in your code?** (2.5 Points)

The reward model uses the GPT model's capability to transform input texts into meaningful embeddings, averages these embeddings or use the hidden state of first token, and then applies a linear transformation to arrive at a scalar score.

This score is intended to reflect the quality or preference of the summary relative to other summaries, based on the training it received. The procedure is identical for both chosen and rejected summaries; what differs is the nature of the text input (i.e., whether the text is from a chosen or rejected summary)

### Load datasets

In [14]:
# Create the comparisons datasets
data_path = comparissions_path
train_pairs = create_comparison_dataset(data_path, "train")
val_pairs = create_comparison_dataset(data_path, "test")

# Make pairwise datasets for training
max_length = 550
train_dataset = PairwiseDataset(train_pairs, tokenizer, max_length=max_length)
val_dataset = PairwiseDataset(val_pairs, tokenizer, max_length=max_length)

# Create the collator to gather batches of pairwise comparisons
data_collator = DataCollatorReward()

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 44155.22it/s]


### Load Model and Tokenizer

Initialize the reward model from the SFT GPT-2 model.

In [53]:
model = GPTRewardModel(CONFIG.output_dir)

tokenizer = AutoTokenizer.from_pretrained(CONFIG.model_name)
tokenizer.pad_token = tokenizer.eos_token

# Freeze the first 70% of the hidden layers of the reward model backbone
layers = model.model.transformer.h
num_layers = len(layers)
num_unfrozen = int(0.3 * num_layers)
for layer in layers[:-num_unfrozen]:
  layer.requires_grad_(False)

### Define Compute metric function (2.5 Points)

In this part you should implement the accuracy of our GPTRewardModel.

In [16]:
def compute_metrics(eval_preds):
    predictions = eval_preds.predictions
    batch_size = predictions.shape[0] // 2
    
    chosen_rewards = predictions[:batch_size]
    rejected_rewards = predictions[batch_size:]
    
    correct_predictions = (chosen_rewards > rejected_rewards).sum().item()
    total_predictions = batch_size
    
    accuracy = correct_predictions / total_predictions
    
    return {"accuracy": accuracy}

### Train

In [58]:
    training_args = TrainingArguments(
      output_dir="rm_checkpoint/",
      num_train_epochs=1,
      logging_steps=5,
      gradient_accumulation_steps=4,
      save_strategy="steps",
      evaluation_strategy="steps",
      per_device_train_batch_size=1,
      per_device_eval_batch_size=1,
      eval_accumulation_steps=1,
      eval_steps=1,
      save_steps=1,
      warmup_steps=100,
      logging_dir="./logs",
      fp16=True,
      bf16=False,
      learning_rate=1e-5,
      save_total_limit=1,
    )

In [59]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        compute_metrics=compute_metrics,
        eval_dataset=val_dataset,
        data_collator=data_collator,
    )

In [60]:
torch.cuda.empty_cache()

In [61]:
trainer.train()
# trainer.save_model(CONFIG.output_dir_rm)

Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
Outputs logits: torch.Size([2, 550, 50257])
Outputs hid.state: 13
Outputs hid.state: torch.Size([2, 550, 768])
O

RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'model.lm_head.weight', 'model.transformer.wte.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [18]:
trainer.train()
trainer.save_model(CONFIG.output_dir_rm)

Step,Training Loss,Validation Loss,Accuracy
250,0.692100,0.679815,0.515030
500,0.684300,0.676564,0.523046


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'model.transformer.wte.weight', 'model.lm_head.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [76]:
# Assuming pairs is your data and tokenizer is initialized
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
pairs = create_comparison_dataset()  # Your function to create pairs

dataset = PairwiseDataset(pairs, tokenizer, max_length=512)
data_collator = DataCollatorReward()

from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size=16, collate_fn=data_collator)

def compute_metrics(eval_preds):
    predictions = eval_preds.predictions
    batch_size = predictions.shape[0] // 2
    
    chosen_rewards = predictions[:batch_size]
    rejected_rewards = predictions[batch_size:]
    
    correct_predictions = (chosen_rewards > rejected_rewards).sum().item()
    total_predictions = batch_size
    
    accuracy = correct_predictions / total_predictions
    
    return {"accuracy": accuracy}


from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

reward_model = GPTRewardModel(model_path="gpt2")

trainer = Trainer(
    model=reward_model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

100%|██████████| 10000/10000 [00:00<00:00, 30642.15it/s]


KeyError: 'chosen_input_ids'

## Section Three: PPO Fine Tuning (25 Points)

### Question 5 (2.5 points)

**What is PPO algorithm? and how it works?**

\# WRITE YOUR ANSWER HERE

### Question 6 (2.5 points)

**Is the PPO algorithm an on-policy or off-policy reinforcement learning algorithm? Explain its functionality within our context (RLHF).**

\# WRITE YOUR ANSWER HERE

### Question 7 (2.5 points)

**Imagine a mini-batch of data has arrived, and we want to optimize the policy of generating summaries to maximize the reward using gradient ascent.**

**Why shouldn't this policy change too much and, in other words, become overoptimized? (Answer based on the respond you provided to the previous question.)**

**What do they do to solve this problem?**

\# WRITE YOUR ANSWER HERE

### Question 8 (2.5 points)

**What is the overestimation problem in the ppo fine-tuning? and why it happens?**

\# WRITE YOUR ANSWER HERE

### Question 9 (2.5 points)

**What potential issue could arise when aligning a language model with human values? What solution has been proposed to address this issue?**

\# WRITE YOUR ANSWER HERE

### Question 10 (2.5 points)

**We know that the objective function of the ppo tuning is as follows:**

$$ \text{objective}(\phi) = \mathbb{E}_{(x,y) \sim D_{\pi_{\phi}^{\text{RL}}}} \left[ r_{\theta}(x, y) - \beta \log \left( \frac{\pi_{\phi}^{\text{RL}}(y \mid x)}{\pi^{\text{SFT}}(y \mid x)} \right) \right] + \gamma \mathbb{E}_{x \sim D_{\text{pretrain}}} \left[ \log(\pi_{\phi}^{\text{RL}}(x)) \right]
 $$

**In the above objective function, the differentiation is with respect to $Φ$, yet the term $r_{\theta}(x, y)$ appears in the function.**

**Explain why $r$ appears in the objective function despite being a function of $\theta$ and why its derivative is not zero.**

\# WRITE YOUR ANSWER HERE

### Question 11 (5 points)

**Another term present in the objective function is $\beta \log \left( \frac{\pi_{\phi}^{\text{RL}}(y \mid x)}{\pi^{\text{SFT}}(y \mid x)} \right)$, which is the KLD between the initial policy distribution and the policy being learned. Explain why this term cannot be calculated directly.**


**For a detailed explanation of how the KLD is calculated in this context, please read this <br>[blog post](http://joschu.net/blog/kl-approx.html). Afterward, provide an explanation of the calculation process.**

\# WRITE YOUR ANSWER HERE

### Question 12 (5 points)

**How does DPO improve fine tuning?**

**In DPO, our loss optimizing function to optimize for the policy is:**
$$\text{LDPO}(\pi_\theta; \pi_\text{ref}) = -\mathbb{E}_{(x,y_w,y_l) \sim D}
\left[
\log \sigma
\left(
\beta \log \frac{\pi_\theta(y_w | x)}{\pi_\text{ref}(y_w | x)}
- \beta \log \frac{\pi_\theta(y_l | x)}{\pi_\text{ref}(y_l | x)}
\right)
\right]$$

**Please explain the terms of this loss function and interpret the function.**


**Now, please calculate the gradient of this loss function and Intuitively explain each term in the gradient of loss function.**

\# WRITE YOUR ANSWER HERE

### Run PPO Fine Tuning (optional)

Because of the limitations of Google Colab, If you have access to an extra  GPU you can run below code for ppo fine tuning.

In [ ]:
%pip install -U git+https://github.com/CarperAI/trlx.git

In [ ]:
import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer

import trlx
from trlx.data.configs import (
    ModelConfig,
    OptimizerConfig,
    SchedulerConfig,
    TokenizerConfig,
    TrainConfig,
    TRLConfig,
)
from trlx.models.modeling_ppo import PPOConfig

In [ ]:
config = TRLConfig(
    train=TrainConfig(
        seq_length=550,
        epochs=50,
        total_steps=100000,
        batch_size=4,
        checkpoint_interval=10000,
        eval_interval=200,
        pipeline="PromptPipeline",
        trainer="AcceleratePPOTrainer",
    ),
    model=ModelConfig(
        model_path="gpt2",
        num_layers_unfrozen=8,
    ),
    tokenizer=TokenizerConfig(
        tokenizer_path="gpt2",
        truncation_side="right",
    ),
    optimizer=OptimizerConfig(
        name="adamw",
        kwargs={
            "lr": 5.0e-6,
            "betas": [0.9, 0.999],
            "eps": 1.0e-8,
            "weight_decay": 0.01,
        },
    ),
    scheduler=SchedulerConfig(
        name="cosine_annealing",
        kwargs={
            "T_max": 100000,
            "eta_min": 5.0e-6,
        },
    ),
    method=PPOConfig(
        name="PPOConfig",
        num_rollouts=128,
        chunk_size=16,
        ppo_epochs=4,
        init_kl_coef=0.1,
        target=6,
        horizon=10000,
        gamma=1,
        lam=0.95,
        cliprange=0.2,
        cliprange_value=0.2,
        vf_coef=0.2,
        scale_reward=None,
        ref_mean=None,
        ref_std=None,
        cliprange_reward=10,
        gen_kwargs={
            "max_new_tokens": 50,
        },
    ),
)


In [ ]:
REWARD_CHECKPOINT_PATH = ""
SFT_MODEL_PATH = ""

In [ ]:
from typing import List
def get_scores(samples: List[str]):
  scores_list = []
  batch_size = 2
  for i in range(0, len(samples), batch_size):
    sub_samples = samples[i : i + batch_size]
    sub_samples = ["<|startoftext|>" + chosen + "<|endoftext|>" for chosen in sub_samples]
    encodings_dict = rw_tokenizer(
        sub_samples,
        truncation=True,
        max_length=config.train.seq_length,
        padding="max_length",
        return_tensors="pt",
        )
    input_ids = encodings_dict["input_ids"].to(rw_device)
    attn_masks = encodings_dict["attention_mask"].to(rw_device)
    input_ids = input_ids.repeat(2, 1)
    attn_masks = attn_masks.repeat(2, 1)
    with torch.no_grad():
      sub_scores = rw_model(input_ids=input_ids, attention_mask=attn_masks)
    scores_list.append(sub_scores["chosen_end_scores"])
  scores = torch.cat(scores_list, dim=0)
  return scores

In [ ]:
rw_tokenizer = AutoTokenizer.from_pretrained("gpt2")
rw_tokenizer.pad_token = rw_tokenizer.eos_token
rw_model = GPTRewardModel(SFT_MODEL_PATH)
rw_model.load_state_dict(torch.load(REWARD_CHECKPOINT_PATH), strict=False)
rw_model.half()
rw_model.eval()
rw_model.to(CONFIG.device)

In [ ]:
def get_prompt_dataset(prompts, max_length):
  formatted_prompts = []
  for i in tqdm(range(len(prompts))):
    tmp = tokenizer.decode(
        tokenizer(
              prompts[i].split("TL;DR:")[0],
              truncation=True,
              max_length=max_length - 5,
              add_special_tokens=False,
            )["input_ids"],
            skip_special_tokens=True,
            ).strip()
    tmp = tmp + "\nTL;DR:"
    tmp = tokenizer.decode(
          tokenizer(tmp, truncation=True, max_length=max_length, add_special_tokens=False)["input_ids"],
          skip_special_tokens=True,
        ).strip()
    formatted_prompts.append(tmp)
  return formatted_prompts

In [ ]:
def reward_fn(samples: List[str], **kwargs):
  original_samples = [text.split("TL;DR:")[0] + "TL;DR: " for text in samples]
  original_samples = [text + post_summary_dict[text.strip()] for text in original_samples]
  original_scores = get_scores(original_samples)
  scores = get_scores(samples)
  norms_scores = scores - original_scores
  return norms_scores

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.tokenizer.tokenizer_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
max_length_input = config.train.seq_length - config.method.gen_kwargs["max_new_tokens"]

# Load the dataset
dataset = load_dataset("carperai/openai_summarize_tldr")

# Get the total number of samples in the training dataset
total_samples = len(dataset["train"])

# Generate random indices for selecting samples
random_indices = random.sample(range(total_samples), 5000)

# Select samples using random indices
train_set = [(dataset["train"][index]["prompt"], dataset["train"][index]["label"]) for index in random_indices]
val_set = [(sample["prompt"], sample["label"]) for sample in dataset["valid"]]

# Split contents into summaries and labels
train_posts, train_summaries = zip(*train_set)
val_posts, val_summaries = zip(*val_set)

# Get the OpenAI summaries
post_summary_dict = {}
train_prompts = get_prompt_dataset(train_posts, max_length_input)
for i in range(len(train_prompts)):
  post_summary_dict[train_prompts[i]] = train_summaries[i]
val_prompts = get_prompt_dataset(val_posts, max_length_input)
for i in range(len(val_prompts)):
  post_summary_dict[val_prompts[i]] = val_summaries[i]

In [ ]:
trainer = trlx.train(
    reward_fn=reward_fn,
    prompts=train_prompts,
    eval_prompts=val_prompts[0:1000],  # sampling 1000 validation prompts for evaluation speed in training
    config=config,
  )

# Quantization (26 points)

Quantization is a technique used to reduce the precision of neural network weights and activations, typically from floating-point to a lower-bit representation, such as 8-bit or 4-bit integers. The primary goal of quantization is to reduce the memory footprint and computational requirements of deep learning models, enabling the loading of larger models that would normally not fit into available memory, and speeding up the inference process.

## A simple example (2 points)

Let's see what happens when a we quantize a 32-bit floating-point number.

In [ ]:
# Import neccesary libraries
import numpy as np
import matplotlib.pyplot as plt

Defining two functions which responsible for quantizing and dequantizing the input number:

In [ ]:
def quantize(value, num_bits=4):
    quantized_value = np.round(value * (2**(num_bits - 1) - 1))
    return int(quantized_value)

def dequantize(quantized_value, num_bits=4):
    value = quantized_value / (2**(num_bits - 1) - 1)
    return float(value)

Consider the value `0.326`, the quantized values in 4 and 8 bits are:

In [ ]:
q_4bit = quantize(value=0.326, num_bits=4)
q_8bit = quantize(value=0.326, num_bits=8)

print(f'4-bit: {q_4bit}')
print(f'8-bit: {q_8bit}')

And if we dequantize it to original full precision values we would have:

In [ ]:
print(f'4-bit: {dequantize(quantized_value=q_4bit, num_bits=4)}')
print(f'8-bit: {dequantize(quantized_value=q_8bit, num_bits=8)}')

8-bit quantization preserves the original precision with very little degradationa and 4-bit quantization does incur more precision loss, but the level of loss can still be tolerated for many applications.

To understand the precision loss from 4-bit and 8-bit quantization, plot the function $y = x^2$ in the range of $[-1, 1]$, and compare the original values to the values obtained after quantization and dequantization for both 4-bit and 8-bit cases. (2 points)

In [ ]:
# WRITE YOUR CODE HERE

## QLoRA: Efficient Finetuning of Quantized LLMs (24 points)

Now, it's time to see the magic of quantization. We are going to fine-tune Mistral 7B model based on the method proposed by [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314).

### Prerequisite

Installing and importing libraries which we will need later.

In [ ]:
!pip install -q accelerate==0.29.3
!pip install -q bitsandbytes==0.43.1
!pip install -q trl==0.8.6
!pip install -q peft==0.10.0
!pip install -q transformers==4.40.0

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import torch

### Hugging Face Login

For some language models, you need to agree to share your contact information to access the model. Mistral 7B is one of them. The steps you should take are as follows:

1.   Create a Gugging Face account if you don't have one.
2.   From Settings > Access Tokens, generate a new token.
3.   From [this link](https://huggingface.co/mistralai/Mistral-7B-v0.1) agree to access the repository.

Now, run the code below to login to your account.

In [ ]:
from huggingface_hub import login
access_token_read = # your api token here
login(token = access_token_read)

### The Model (6 points)

To load the model, first we need a quantization configuration to set how we want to quantize the model. We are going to use `BitsAndBytesConfig` to achieve our goal. `BitsAndBytesConfig` is the easiest option for quantizing a model to 8 and 4-bit.

Define the quantization configuration in the cell below based on the fact that we are looking for 4-bit quantization. Also, QLoRA paper proposed two techniques: 4-bit NormalFloat(NF4) quantization and Double Quantization. Set these two setup in our configuration too. To speedup the computation try to set computational type as bf16. (2 points)

Try to explain what is the Double Quantization technique. You can use the provided paper to read about it. (4 points)

\# WRITE YOUR ANSWER HERE

In [ ]:
model_name = 'mistralai/Mistral-7B-v0.1'

quantization_config = # WRITE YOUR CODE HERE

Now, let's load the model.

In [ ]:
device_map = {"": 0}
model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=quantization_config,
                    device_map=device_map,
                    use_cache = False)

Now, we have the quantized model on the memory. You can try to load the model without quantization into memory to see if it is possible.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

### Generate text using the pre-trained model (1 point)

In this part we are going to test the pre-trained model to see its capabilities.

In [ ]:
# Function which generate outputs corresponding to input prompts
def generate_output(model, inputs, max_length=50):

    tokenized_inputs = tokenizer(inputs, padding=True, return_tensors="pt").to('cuda')
    output = model.generate(
        input_ids=tokenized_inputs["input_ids"],
        attention_mask=tokenized_inputs["attention_mask"],
        max_new_tokens=max_length,
        repetition_penalty=1.5,
        early_stopping=False,
        eos_token_id=tokenizer.eos_token_id,
    )

    text = tokenizer.batch_decode(output, skip_special_tokens=True)
    return text

Using the `generate_output` function try to produce output of the pre-trained model for the following prompts: (1 point)

* "What's up?"
* 'Hello, How are you?'
* 'hello, Can you help me?'
* 'what is the capital of France?'
* 'Hi! what do you do for a living?'



In [ ]:
# WRITE YOUR CODE HERE

### The Dataset (4 points)

We loaded a 7B LLM into the memory and succeeded to inference from the model. Now, using [this dataset](https://huggingface.co/datasets/SAGI-1/Greetings_DPO_dataset_V1) we want to fine-tune our pre-trained model.

After loading the dataset, choose 75 samples of it. We would like to backpropagate only the tokens of the completion and not the prompt itself. In order to do this, use `DataCollatorForCompletionOnlyLM` (check the examples provided in [this link](https://huggingface.co/docs/trl/en/sft_trainer#train-on-completions-only)). Also, you can ignore the `rejected` column.

In [ ]:
# WRITE YOUR CODE HERE

### Fine-tuning (8 points)

We want to fine-tune the model with the help of [LoRA](https://arxiv.org/abs/2106.09685). Try to create a LoRA configuration with rank and alpha parameter both equal to 12. (3 points)

In [ ]:
import peft
from peft import LoraConfig, get_peft_model

target_modules = ["q_proj", "v_proj"]
lora_config = # WRITE YOUR CODE HERE

We are going to use SFTtrainer from trl library. In order to work within memory constraints, try to set the batch size and the number of update steps to accumulate the gradients equal to 1. Also, don't forget to enable the `gradient_checkpointing`. (5 points)

In [ ]:
# WRITE YOUR CODE HERE

### Check the fine-tuned model (5 points)

Try to check the fine-tuned model on the five aformentioned prompts which we test earlier. Compare you results of fine-tuned and pre-trained model.

\# WRITE YOUR ANSWER HERE

In [ ]:
# WRITE YOUR CODE HERE

In case you have memory problem try `del` statement for the variables you don't need. The code below also could be beneficial.

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

# Instruct Tuning (4 points)

What we done earlier was somehow instruct tuning in which we fine-tune the pre-trained model to follow the instructions.However, with 75 samples we can't do anything big. As you probably know, the pre-training objective for auto-regressive models is to simply predict the next word. Therefore, they are not optimized to answer to your prompts, rather they try continue the text you provided.

In many practical settings (like chatbots) we need the model to follow the instructions provided by the user. So, huge datasets consisting of prompts and their corresponding answer are used to fine-tune these models for interactive use cases.

Using the quantization technique we used earlier try to load the quantized version of [Mistral-7B-Instruct](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) which have been optimized for following the instructions. Check the five samples we used earlier and compare your results.

Try to check the provided link to see how you should format the prompt to leverage instruction fine-tuning.

In [ ]:
instruct_model = 'mistralai/Mistral-7B-Instruct-v0.1'
# WRITE YOUR CODE HERE

\# WRITE YOUR ANSWER HERE

# Evaluation (25 points)

## Evaluating text using a language model

One way of evaluating text generation is by using a language model.
In this assignment, we want to use BERTScore to compare the similarity of sentences.
After reading [BERTScore paper](https://arxiv.org/abs/1904.09675) answer the following questions.

### 1)
How does BERTScore evaluate semantic equivalence and how is it better than n-gram based metrics like BLEU? (5 points)

\# WRITE YOUR ANSWER HERE

### 2)
Implement BERTScore recall $R_{BERT}$ in the paper. You don't need to bother with the Importance Weighting. We use a more modern model called deberta which is better than bert in many ways. Then visualize similarity matrix. You can use the visualization code given to you. (7 points)

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")
model = AutoModel.from_pretrained("microsoft/deberta-v3-small")

def cosine_similarity(a, b):
    return # WRITE YOUR CODE HERE (implement using pytorch CosineSimilarity)

def bert_score(reference, candidate):


    # WRITE YOUR CODE HERE (Tokenize the input text)
    ref_tokens = "tokenize reference sentence"
    candidate_tokens = "you should know what to do here"

    model.eval()
    with torch.no_grad():
        # WRITE YOUR CODE HERE (get the embeddings please)
        pass


    similarities = "The similarities matrix"

    R_BERT = """You can see max in the BERTScore formula in the paper. Then what are you waiting for? Implement the thing.
            Pay attention to the fact that you are computing recall. It's important to be careful about
              which axis you are getting the max on."""
    bertscore = R_BERT.mean()


    return bertscore, similarities


In [ ]:
reference = "Cats are cute"
candidate = "Cats are annoying"

bertscore, sim_matrix = bert_score(
    reference, candidate)

print(bertscore)
print(sim_matrix)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def heatmap(
    matrix,
    hide_spines=False,
    hide_ticks=False,
    figsize=None,
    cmap=None,
    colorbar=True,
    row_names=None,
    column_names=None,
    column_name_rotation=45,
    cell_values=True,
    cell_fmt=".2f",
    cell_font_size=None,
):

    if row_names is not None and len(row_names) != matrix.shape[0]:
        raise AssertionError(
            f"len(row_names) (got {len(row_names)})"
            " should be equal to number of"
            " rows in the input "
            f" array (expect {matrix.shape[0]})."
        )

    if column_names is not None and len(column_names) != matrix.shape[1]:
        raise AssertionError(
            f"len(column_names)"
            " (got {len(column_names)})"
            " should be equal to number of"
            " columns in the"
            f" input array (expect {matrix.shape[1]})."
        )

    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)

    if cmap is None:
        cmap = plt.cm.viridis

    if figsize is None:
        figsize = (len(matrix) * 1.5, len(matrix) * 1.5)

    matshow = ax.matshow(matrix, cmap=cmap)

    if colorbar:
        fig.colorbar(matshow)

    normed_matrix = matrix.astype("float") / matrix.max()

    if cell_values:
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                cell_text = format(matrix[i, j], cell_fmt)

                ax.text(
                    x=j,
                    y=i,
                    size=cell_font_size,
                    s=cell_text,
                    va="center",
                    ha="center",
                    color="black"
                    if normed_matrix[i, j] > np.max(normed_matrix) / 2
                    else "black",
                )

    if row_names is not None:
        tick_marks = np.arange(len(row_names))
        plt.yticks(tick_marks, row_names)

    if column_names is not None:
        tick_marks = np.arange(len(column_names))

        if column_name_rotation:
            plt.xticks(
                tick_marks,
                column_names,
                rotation=column_name_rotation,
                ha="right",
                rotation_mode="anchor",
            )
        else:
            plt.xticks(tick_marks, column_names)

    if hide_spines:
        ax.spines["right"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
    ax.yaxis.set_ticks_position("left")
    ax.xaxis.set_ticks_position("bottom")
    if hide_ticks:
        ax.axes.get_yaxis().set_ticks([])
        ax.axes.get_xaxis().set_ticks([])

    return fig, ax

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap

viridisBig = cm.get_cmap('Blues', 512)
newcmp = ListedColormap(viridisBig(np.linspace(0.0, 0.5, 256)))

fig, ax = heatmap(sim_matrix, column_names=reference.split(), row_names=candidate.split(), cmap=newcmp, figsize=(4, 4))

plt.tight_layout();
plt.show()

### 3)
Using the [Official BERTScore](https://github.com/Tiiiger/bert_score) package. Compare your implementation with the official one and tell us why they differ. (3 points)

In [ ]:
# pip install bert-score

In [ ]:
import torch
from bert_score import score

reference = "Cats are cute"
candidate = "Cats are annoying"

P, R, F1 = score(
    # WRITE YOUR CODE HERE (complete yourself)
)


### 4)
Download datasets "sst2" and "imdb" from huggingface datasets. Both of these datasets are for the same task but their distribution differs. In what ways their distribution differ according to [this paper](https://aclanthology.org/2021.emnlp-main.835/)?

randomly choose ten sentences from negative and ten sentences from positive labels of each dataset (forty sentences in total)

use your implementation of BERTScore and compute similarity of

a) sentences that are in the same dataset and have same label and take average

b) sentences that are in the same dataset and have opposite label and take average

c) sentences that are in different datasets and have same label and take average

d) sentences that are in the different dataset and have opposite label and take average

How these numbers differ and why? (for example for part a you should put each sentence as reference once and compare it with ten candidate sentences including itself, after doing this for all ten sentences you get a hundred scores and then an average.) (10 points)

\# WRITE YOUR ANSWER HERE

# AI Assistant

* How to create a comparison dataset for a text summarization task for reward model in RLHF?
* What is the goal of the pairwise ranking loss in the context of training a reward model?
* How the pairwise ranking loss can avoid the problem of huge score difference between the answers (summaries) during training and why is this useful?